In [1]:
from importlib import reload
import utils
import models_gpu # as models
import numpy as np
from time import time
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger # maybe more
import json

Using TensorFlow backend.


In [2]:
def run_kumar_model():


    BATCH_SIZE = 512
    EPOCHS = 1 # it seemed to converge by this point anyway

    # choose hyperparameters
    num_layers = np.random.randint(2,16) # lstm layers
    hidden_state_size = np.random.randint(100,400)
    dropout_rate = round(np.random.uniform(0,1,size=None), 3)
    embed_vec = np.random.choice(['glove200', 'glove300', 'numberbatch'])
    #sequence_length = np.random.randint(1,100)
    sequence_length = 2
    bidirectional = True
    stateful = False
    trainable_embed = True

    ## start the timer
    start = time()

    # load the corpus
    corpus = utils.AMI_Corpus(seed = 75, embed_vec = embed_vec)

    bi_lstm = models.BiLSTMCRF(corpus,
                               batch_size = BATCH_SIZE,
                               sequence_length = sequence_length,
                               num_layers = num_layers,
                               dropout_rate = dropout_rate,
                               hidden_state_size = hidden_state_size,
                               stateful = stateful, bidirectional = bidirectional, 
                               trainable_embed = trainable_embed)

    bi_lstm.compile()

    ug_train = utils.UtteranceGenerator(corpus, "train", batch_size = BATCH_SIZE, sequence_length = sequence_length, conv= True, kumar = True)
    ug_val = utils.UtteranceGenerator(corpus, "val", batch_size = BATCH_SIZE, sequence_length = sequence_length, conv= True, kumar = True)

    # create keras callbacks
    es = EarlyStopping(monitor='val_loss', patience=5, verbose=0)

    right_now = datetime.now().isoformat() # timestamp
    csv_logger = CSVLogger('logs/kumar_history_' + str(it_no) + ".csv") # log epochs in case I want to look back later

    # note to self, maybe change validation_steps and validation_freq
    history = bi_lstm.model.fit_generator(ug_train, epochs=EPOCHS, verbose=1, callbacks=[es, csv_logger],
                                          validation_data=ug_val)

    results = {"num_layers":num_layers, "hidden_state_size":hidden_state_size,"dropout_rate":dropout_rate, 
               "sequence_length":sequence_length,"embed_vec":embed_vec, "acc":history.history['acc'][-5:], 
               "val_acc":history.history['acc'][-5:], "loss":history.history['loss'][-5:], 
               "val_loss":history.history['val_loss'][-5:], "time":time() - start,"trainable_embed":trainable_embed, 
               "stateful":stateful, "bidirectional":bidirectional}
    
    results = {str(k):str(v) for k,v in results.items()}

    with open("logs/kumar_params_" + str(it_no) + ".json", "w") as f:
        json.dump(results, f)
    
    print("-------------------")

    
    return results

In [3]:
reload(utils)
reload(models)
reults = run_kumar_model(it_no = 0)

Loading corpus from /home/jake_miller/final/nlp-final-project/data
Begin corpus post-processing ...
Splitting corpus into training and test ...
Creating vocabulary from training set ...
Found 8379 unique words.
Building initial embedding matrix ...
(8381, 200)
loading pretrained vectors from glove.6B.200d.txt


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


TypeError: __int__ returned non-int (type NoneType)